<a href="https://colab.research.google.com/github/kanish-dev/Anomaly-Detection-/blob/main/Anomaly_Detection_in_Transactions_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget 'https://raw.githubusercontent.com/kanish-dev/Anomaly-Detection-/main/transaction_anomalies_dataset.csv'

--2023-09-27 05:57:44--  https://raw.githubusercontent.com/kanish-dev/Anomaly-Detection-/main/transaction_anomalies_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88775 (87K) [text/plain]
Saving to: ‘transaction_anomalies_dataset.csv’

transaction_anomali 100%[===================>]  86.69K  --.-KB/s    in 0.02s   

2023-09-27 05:57:45 (5.55 MB/s) - ‘transaction_anomalies_dataset.csv’ saved [88775/88775]



In [ ]:
import pandas as pd
import plotly as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import  classification_report


In [ ]:
# import plotly.express as px
# df = px.data.tips()
# # Here we use a column with categorical data
# fig = px.histogram(df, x="day")
# fig.show()

In [ ]:
data = pd.read_csv("transaction_anomalies_dataset.csv")
data.head(20)

,Transaction_ID,Transaction_Amount,Transaction_Volume,Average_Transaction_Amount,Frequency_of_Transactions,Time_Since_Last_Transaction,Day_of_Week,Time_of_Day,Age,Gender,Income,Account_Type
0,TX0,1024.835708,3,997.234714,12,29,Friday,06:00,36,Male,1436074,Savings
1,TX1,1013.952065,4,1020.210306,7,22,Friday,01:00,41,Female,627069,Savings
2,TX2,970.956093,1,989.496604,5,12,Tuesday,21:00,61,Male,786232,Savings
3,TX3,1040.822254,2,969.522480,16,28,Sunday,14:00,61,Male,619030,Savings
4,TX4,998.777241,1,1007.111026,7,7,Friday,08:00,56,Female,649457,Savings
5,TX5,969.987307,1,1018.948796,6,20,Thursday,14:00,24,Male,872843,Savings
6,TX6,973.317560,2,999.889443,5,4,Tuesday,07:00,21,Male,1105889,Savings
7,TX7,931.103035,4,992.937667,10,29,Sunday,17:00,43,Male,502283,Savings
8,TX8,1036.923329,4,1003.427366,18,23,Thursday,07:00,38,Female,323247,Savings
9,TX9,1013.174302,4,1032.835422,14,25,Thursday,12:00,58,Female,1278732,Current


In [ ]:
data.isnull().sum()
# as we see there are no null values in data set

Transaction_ID                 0
Transaction_Amount             0
Transaction_Volume             0
Average_Transaction_Amount     0
Frequency_of_Transactions      0
Time_Since_Last_Transaction    0
Day_of_Week                    0
Time_of_Day                    0
Age                            0
Gender                         0
Income                         0
Account_Type                   0
dtype: int64

In [ ]:
data.info()
# lets have the column insights from =>.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Transaction_ID               1000 non-null   object 
 1   Transaction_Amount           1000 non-null   float64
 2   Transaction_Volume           1000 non-null   int64  
 3   Average_Transaction_Amount   1000 non-null   float64
 4   Frequency_of_Transactions    1000 non-null   int64  
 5   Time_Since_Last_Transaction  1000 non-null   int64  
 6   Day_of_Week                  1000 non-null   object 
 7   Time_of_Day                  1000 non-null   object 
 8   Age                          1000 non-null   int64  
 9   Gender                       1000 non-null   object 
 10  Income                       1000 non-null   int64  
 11  Account_Type                 1000 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 93.9+ KB


In [ ]:
data.describe()
# .describe() => helps to gather the stats of the data

,Transaction_Amount,Transaction_Volume,Average_Transaction_Amount,Frequency_of_Transactions,Time_Since_Last_Transaction,Age,Income
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,1038.122511,2.498000,1000.682506,12.078000,15.341000,40.641000,8.948238e+05
std,283.580055,1.115006,20.632334,4.245225,8.361258,13.819953,3.453562e+05
min,849.024392,1.000000,939.081423,5.000000,1.000000,18.000000,3.001590e+05
25%,966.028796,1.000000,986.800556,8.000000,8.000000,29.000000,5.917308e+05
50%,1002.118678,3.000000,1000.501902,12.000000,16.000000,41.000000,8.876645e+05
75%,1033.143657,3.000000,1015.155595,16.000000,22.000000,53.000000,1.178102e+06
max,3227.459108,4.000000,1073.154036,19.000000,29.000000,64.000000,1.499670e+06


In [ ]:
fig_amount = px.histogram(data, x='Transaction_Amount',
                          nbins=20,
                          title='Distribution of Transaction Amount')
fig_amount.show()

In [ ]:
fig_box_amount = px.box(data,
                 x="Account_Type",
                 y='Transaction_Amount',
                 title = 'Transaction Amount by Account Type')
fig_box_amount.show()

In [ ]:
fig_box_amount = px.box(data,
                 x="Gender",
                 y='Income',
                 title = 'Income to Gender Ratio')
fig_box_amount.show()

In [ ]:
fig_scatter_avg_amount_age = px.scatter(data,x='Income',
                                        y="Average_Transaction_Amount",
                                        color = "Gender",
                                        title = "Average Transaction Amount by Age",
                                        trendline='ols' )
fig_scatter_avg_amount_age.show()

In [ ]:
fig_scatter_avg_amount_age = px.scatter(data,x='Age',
                                        y="Average_Transaction_Amount",
                                        color = "Account_Type",
                                        title = "Average Transaction Amount by Age",
                                        trendline='ols')
fig_scatter_avg_amount_age.show()

In [ ]:
fig_day_of_week = px.bar(data,x='Day_of_Week',
                         title="Count of Transactions by Day of the Week")
fig_day_of_week.show()

In [ ]:
# corelation heatmap
correlation_matrix = data.corr()
fig_corr_heatmap  = px.imshow(correlation_matrix,title='Correlation Heatmap')
fig_corr_heatmap.show()

<ipython-input-16-24f10122cef1>:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
# how to visualize the anomalies of the data
mean_amount = data['Transaction_Amount'].mean()
std_amount = data['Transaction_Amount'].std()
print(mean_amount)
print(std_amount)
# define the anamoly thereshold(2 standard deviantions from mean)
anamaly_threshold = mean_amount+2*std_amount
print(anamaly_threshold)

1038.1225107502053
283.58005471535563
1605.2826201809166


In [ ]:
# flag the anamolies
data["Is_Anomaly"] = data['Transaction_Amount']>anamaly_threshold
data['Is_Anomaly']

fig_anomalies = px.scatter(data,
                           x='Transaction_Amount',
                           y='Average_Transaction_Amount',
                           color='Is_Anomaly', title="Anamolies in Transaction Amount")
fig_anomalies.update_traces(maker=dict(size=12),selector=dict(mode='makers',maker_size=1))
fig_anomalies.show()

In [ ]:
# calculating the no of anamolies
num_anomalies = data['Is_Anomaly'].sum()


# total no of instances in dtaset
num_instances = data.shape[0]


# anamoly ratio is
anamoly_ratio = num_anomalies / num_instances

print(anamoly_ratio)

0.02


In [ ]:
# here is the traiing of the machine to detect anomalies
relavant_features = ['Transaction_Amount',
                     'Average_Transaction_Amount',
                     'Frequency_of_Transactions']
x= data[relavant_features]
y = data['Is_Anomaly']

# spliliting of the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = IsolationForest(contamination=0.02,random_state=42)
model.fit(x_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



IsolationForest(contamination=0.02, random_state=42)

In [ ]:
# Predict anomalies on the test set
y_pred = model.predict(x_test)

# Convert predictions to binary values (0: normal, 1: anomaly)
y_pred_binary = [1 if pred == -1 else 0 for pred in y_pred]

# Evaluate the model's performance
report = classification_report(y_test, y_pred_binary, target_names=['Normal', 'Anomaly'])
print(report)


              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00       196
     Anomaly       1.00      1.00      1.00         4

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
relavant_features = ['Transaction_Amount',
                     'Average_Transaction_Amount',
                     'Frequency_of_Transactions']
user_inputs = []

for feature in relavant_features:
  user_input = input(f"Enter the value for '{feature}':")
  user_inputs.append(user_input)

user_df = pd.DataFrame([user_inputs],columns=relavant_features)

user_anomaly_pred = model.predict(user_df)


predicted_op = 1 if user_anomaly_pred == -1 else 0
if predicted_op == 1:
  print("Alert !!! we fouund an Anomaly")
else:
  print("This is a NORMAL transaction ")



Enter the value for 'Transaction_Amount':1000000
Enter the value for 'Average_Transaction_Amount':8000
Enter the value for 'Frequency_of_Transactions':20
Alert !!! we fouund an Anomaly
